# Project : Data Cleaning - Sales Dataset

## Inspecing Data

In [1]:
import os
import pandas as pd

In [2]:
pd.options.display.max_columns = 30
df = pd.read_csv("all_data.csv", low_memory=False)
df

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,09/17/19 20:56,"840 Highland St, Los Angeles, CA 90001"
186846,259354,iPhone,1,700,09/01/19 16:00,"216 Dogwood St, San Francisco, CA 94016"
186847,259355,iPhone,1,700,09/23/19 07:39,"220 12th St, San Francisco, CA 94016"
186848,259356,34in Ultrawide Monitor,1,379.99,09/19/19 17:30,"511 Forest St, San Francisco, CA 94016"


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186850 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 8.6+ MB


In [34]:
df.Product[0]

'USB-C Charging Cable'

In [5]:
df.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


In [6]:
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price Each', 'Order Date',
       'Purchase Address'],
      dtype='object')

## Rename the column "Price Each" to "Price/unit"

In [7]:
df.rename(columns={"Price Each":"Price/unit"},inplace = True)

In [8]:
# Confirm the change
df.columns

Index(['Order ID', 'Product', 'Quantity Ordered', 'Price/unit', 'Order Date',
       'Purchase Address'],
      dtype='object')

## Remove rows that contain empty cells

In [9]:
df.isnull().sum()

Order ID            545
Product             545
Quantity Ordered    545
Price/unit          545
Order Date          545
Purchase Address    545
dtype: int64

In [10]:
df.dropna(inplace = True)

In [11]:
#Verifing empty cells exist
df.isnull().sum()

Order ID            0
Product             0
Quantity Ordered    0
Price/unit          0
Order Date          0
Purchase Address    0
dtype: int64

In [12]:
#Verifing empty cells exist
nan_df = df[df.isna().any(axis=1)]
display(nan_df.head())

,Order ID,Product,Quantity Ordered,Price/unit,Order Date,Purchase Address


## Data Type Conversion: Ensure that each column has the correct data type.

In [13]:
for col_name in df.columns:
    product_type = df[col_name].dtype
    print("The data type of the {0} column is:{1}".format(col_name, product_type))

The data type of the Order ID column is:object
The data type of the Product column is:object
The data type of the Quantity Ordered column is:object
The data type of the Price/unit column is:object
The data type of the Order Date column is:object
The data type of the Purchase Address column is:object


In [14]:
#Some data cannot be coverted to int datatype, below code removes the row which cannot be coverted 
def is_numeric(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

# Create a mask to identify rows with parsing errors
mask = df['Quantity Ordered'].apply(lambda x: not is_numeric(x))

# Get the positions of rows with parsing errors
error_positions = df[mask].index

# Drop rows with parsing errors from the DataFrame
df.drop(error_positions, inplace=True)

# Now, 'df' will be a DataFrame with the row causing the ValueError removed.

In [15]:
#Converts 'Quantity Ordered' column into numeric
df['Quantity Ordered'] = pd.to_numeric(df['Quantity Ordered'])

In [16]:
#Verifing data type of 'Quantity Ordered'
data_types = df['Quantity Ordered'].dtypes
print(data_types)

int64


In [17]:
#Converts 'Price/unit' column into numeric
df['Price/unit'] = pd.to_numeric(df['Price/unit'])

In [18]:
#Verifing data type of 'Quantity Ordered'
data_types = df['Price/unit'].dtypes
print(data_types)

float64


In [19]:
#Converts 'Order ID' column into numeric
df['Order ID'] = pd.to_numeric(df['Order ID'])

In [20]:
#Verifing data type of 'Order ID'
data_types = df['Order ID'].dtypes
print(data_types)

int64


## Parsing Dates: Convert the 'Order Date' column to a datetime data type to facilitate date-based analysis.

In [22]:
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [23]:
data_types = df['Order Date'].dtypes
print(data_types)

datetime64[ns]


## Extracting Address Information : Extract relevant address information (city & state) from the 'Purchase Address'

In [24]:
def get_city(address):
    return address.split(",")[1].strip(" ")

def get_state(address):
    return address.split(",")[2].split(" ")[1]

df['City'] = df['Purchase Address'].apply(lambda x: f"{get_city(x)}")
df['State'] = df['Purchase Address'].apply(lambda x: f"{get_state(x)}")
df.head()

,Order ID,Product,Quantity Ordered,Price/unit,Order Date,Purchase Address,City,State
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",Dallas,TX
2,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",Boston,MA
3,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,CA
4,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,CA
5,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",Los Angeles,CA


## Add month column

In [25]:
df['Month'] = df['Order Date'].dt.month
print(df.head())

   Order ID                     Product  Quantity Ordered  Price/unit  \
0    176558        USB-C Charging Cable                 2       11.95   
2    176559  Bose SoundSport Headphones                 1       99.99   
3    176560                Google Phone                 1      600.00   
4    176560            Wired Headphones                 1       11.99   
5    176561            Wired Headphones                 1       11.99   

           Order Date                      Purchase Address         City  \
0 2019-04-19 08:46:00          917 1st St, Dallas, TX 75001       Dallas   
2 2019-04-07 22:30:00     682 Chestnut St, Boston, MA 02215       Boston   
3 2019-04-12 14:38:00  669 Spruce St, Los Angeles, CA 90001  Los Angeles   
4 2019-04-12 14:38:00  669 Spruce St, Los Angeles, CA 90001  Los Angeles   
5 2019-04-30 09:27:00     333 8th St, Los Angeles, CA 90001  Los Angeles   

  State  Month  
0    TX      4  
2    MA      4  
3    CA      4  
4    CA      4  
5    CA      4  


## Removing duplicates

In [26]:
value_counts = df['Order ID'].value_counts()
print(value_counts)

160873    5
178158    4
277875    4
196615    4
304802    4
         ..
158177    1
158178    1
158179    1
158180    1
259357    1
Name: Order ID, Length: 178437, dtype: int64


In [27]:
#Removing duplicates in Order ID, since Order ID is always unique
df.drop_duplicates(subset="Order ID", keep="first", inplace=True)

In [28]:
#Verifying column "Order ID" consists duplicates
value_counts = df['Order ID'].value_counts()
print(value_counts)

176558    1
197960    1
197952    1
197953    1
197954    1
         ..
156958    1
156959    1
156960    1
156961    1
259357    1
Name: Order ID, Length: 178437, dtype: int64


## Separating date and time from column "Order Date"

In [31]:
df["Date"] = df["Order Date"].dt.date
df["Time"] = df["Order Date"].dt.time
# Droppping column "Order Date"
df.drop("Order Date", axis=1, inplace=True)

In [37]:
print(df)

        Order ID                     Product  Quantity Ordered  Price/unit  \
0         176558        USB-C Charging Cable                 2       11.95   
2         176559  Bose SoundSport Headphones                 1       99.99   
3         176560                Google Phone                 1      600.00   
5         176561            Wired Headphones                 1       11.99   
6         176562        USB-C Charging Cable                 1       11.95   
...          ...                         ...               ...         ...   
186845    259353      AAA Batteries (4-pack)                 3        2.99   
186846    259354                      iPhone                 1      700.00   
186847    259355                      iPhone                 1      700.00   
186848    259356      34in Ultrawide Monitor                 1      379.99   
186849    259357        USB-C Charging Cable                 1       11.95   

                               Purchase Address           City 

## Adding column "Total Cost" (Total Cost = Quantity Ordered * Price/unit)

In [38]:
df["Total Cost"] = df["Quantity Ordered"] * df["Price/unit"]
print(df)

        Order ID                     Product  Quantity Ordered  Price/unit  \
0         176558        USB-C Charging Cable                 2       11.95   
2         176559  Bose SoundSport Headphones                 1       99.99   
3         176560                Google Phone                 1      600.00   
5         176561            Wired Headphones                 1       11.99   
6         176562        USB-C Charging Cable                 1       11.95   
...          ...                         ...               ...         ...   
186845    259353      AAA Batteries (4-pack)                 3        2.99   
186846    259354                      iPhone                 1      700.00   
186847    259355                      iPhone                 1      700.00   
186848    259356      34in Ultrawide Monitor                 1      379.99   
186849    259357        USB-C Charging Cable                 1       11.95   

                               Purchase Address           City 

In [39]:
data_types = df['Total Cost'].dtypes
print(data_types)

float64


## Final cleaned dataset

In [40]:
df

,Order ID,Product,Quantity Ordered,Price/unit,Purchase Address,City,State,Month,Date,Time,Total Cost
0,176558,USB-C Charging Cable,2,11.95,"917 1st St, Dallas, TX 75001",Dallas,TX,4,2019-04-19,08:46:00,23.90
2,176559,Bose SoundSport Headphones,1,99.99,"682 Chestnut St, Boston, MA 02215",Boston,MA,4,2019-04-07,22:30:00,99.99
3,176560,Google Phone,1,600.00,"669 Spruce St, Los Angeles, CA 90001",Los Angeles,CA,4,2019-04-12,14:38:00,600.00
5,176561,Wired Headphones,1,11.99,"333 8th St, Los Angeles, CA 90001",Los Angeles,CA,4,2019-04-30,09:27:00,11.99
6,176562,USB-C Charging Cable,1,11.95,"381 Wilson St, San Francisco, CA 94016",San Francisco,CA,4,2019-04-29,13:03:00,11.95
...,...,...,...,...,...,...,...,...,...,...,...
186845,259353,AAA Batteries (4-pack),3,2.99,"840 Highland St, Los Angeles, CA 90001",Los Angeles,CA,9,2019-09-17,20:56:00,8.97
186846,259354,iPhone,1,700.00,"216 Dogwood St, San Francisco, CA 94016",San Francisco,CA,9,2019-09-01,16:00:00,700.00
186847,259355,iPhone,1,700.00,"220 12th St, San Francisco, CA 94016",San Francisco,CA,9,2019-09-23,07:39:00,700.00
186848,259356,34in Ultrawide Monitor,1,379.99,"511 Forest St, San Francisco, CA 94016",San Francisco,CA,9,2019-09-19,17:30:00,379.99
